In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('max_rows',1000)
import os
from glob import iglob

In [10]:
# Load all structural p-values across both analyses
df = pd.read_csv("/Volumes/sivleyrm/pdbmap/results/specialK_analysis_2016-05-06/clinvar/pathogenic_K_summary.txt",delimiter='\t')

In [11]:
print "Number of structures evaluated:"
print len(df)
print "Number of variants evluated:"
print "%.0f"%np.sum(df["N"])

from qvalue import estimate
THRESH = 0.1

df.ix[~df[ "Kp"].isnull(), "Kq"], K_lam, K_pi = estimate(df.ix[~df[ "Kp"].isnull(), "Kp"].values)

print "\nProteins passing an FDR of %.0f%% for the unweighted analysis"%(THRESH*100)
print "Significant:   %4d"%( df['Kq']<THRESH).sum()
print "    Clustered: %4d"%((df['Kq']<THRESH) & (df["Kz"]>0)).sum()
print "    Dispersed: %4d"%((df['Kq']<THRESH) & (df["Kz"]<0)).sum()

Number of structures evaluated:
453
Number of variants evluated:
4827

Proteins passing an FDR of 10% for the unweighted analysis
Significant:     88
    Clustered:   88
    Dispersed:    0


In [13]:
df.to_csv("/Volumes/sivleyrm/pdbmap/results/ripleysK_results/pdb_clinvar_missense_univariate.txt",
            header=False,index=False,sep="\t",na_rep="\N")

In [ ]:
# P-Value Distributions
fig,ax = plt.subplots(1,1,figsize=(20,6))
plt.suptitle("P-Value Distributions for ExAC Univariate K",fontsize=25,y=1.02)
ax.set_xlabel("Un-Weighted K p-value",fontsize=20)
# ax[0].hist(df["Kzp"],bins=np.arange(0,1.02,0.02),color="darkblue",normed=True)
ax.hist(df["Kp"],bins=np.arange(0,1.02,0.02),color="darkred",alpha=0.8,normed=True)
ax.plot(K_lam,K_pi,lw=3,c='cornflowerblue')
plt.tight_layout()
plt.show()

In [ ]:
dfv = pd.DataFrame(df[["Kp","Kz"]].values,columns=["p","z"])
dfv['q'] = estimate(dfv["p"].values)[0]
dfv["w"] = "un-weighted"
dfv['dummy'] = ""

fig,ax = plt.subplots(1,3,figsize=(20,10),sharey=True)
sns.violinplot(x='dummy',y="z",hue='w',data=dfv,ax=ax[0],cut=1,orient='v',inner='quart')
dfv1 = dfv[dfv["p"]<0.05]
if not dfv1.empty:
    split = (dfv1["w"]=="weighted").sum() > 0
    sns.violinplot(x='dummy',y="z",hue='w',data=dfv1,ax=ax[1],orient='v',split=split,cut=0,inner='quart')
    sns.stripplot( x='dummy',y="z",hue='w',data=dfv1,ax=ax[1],orient='v',split=True,jitter=True,s=7,lw=1,edgecolor='white')
dfv1 = dfv[dfv["q"]<THRESH]
if not dfv1.empty:
    split = (dfv1["w"]=="weighted").sum() > 0
    sns.violinplot(x='dummy',y="z",hue='w',data=dfv1,ax=ax[2],orient='v',split=split,cut=0,inner='quart')
    sns.stripplot( x='dummy',y="z",hue='w',data=dfv1,ax=ax[2],orient='v',split=True,jitter=True,s=7,lw=1,edgecolor='white')
ax[0].set_xlabel("ALL",fontsize=15)
ax[1].set_xlabel("Nominally Significant (p<0.05)",fontsize=15)
ax[2].set_xlabel("FDR Significant (q<%.2f)"%THRESH,fontsize=15)
ax[0].legend(fontsize=15)
ax[1].legend(fontsize=15)
ax[2].legend(fontsize=15)
plt.ylim([-5,20])
ax[0].axhline(0.,ls='dashed',c='black')
ax[1].axhline(0.,ls='dashed',c='black')
ax[2].axhline(0.,ls='dashed',c='black')
plt.show()

In [ ]:
print "Un-Weighted: FDR-Significant Proteins"
print df.ix[df["Kq"]<0.1,["structid","chain","Kz","Kp","Kq"]]

In [ ]:
# Examine how many COSMIC-significant proteins contain 3+ ClinVar nsSNVs
# In how many of those proteins were ClinVar nsSNVs also significantly clustered?
# These are generated from the end of the univariate COSMIC notebook
csm_sig = ['2SHP', '2OVQ', '4NM6', '4BHW', '1D5R', '4JSP', '2WTK', '2RD0', '1RJB', '3IJ9', '3GT8', '2K60', '2Q7Z', '2Y1M', '1MFV']
ovlp = df[df['structid'].isin(csm_sig)]
print len(csm_sig)-len(ovlp),'of',len(csm_sig),'COSMIC-significant proteins do not contain 3+ ClinVar nsSNVs'
print '\n',ovlp[["structid","chain","R","N","Kz","Kq"]]
print '\n',(ovlp["Kq"]<0.1).sum(),'of',len(ovlp),'of the overlapping proteins are FDR-significant in ClinVar'
print '\n',ovlp.ix[ovlp["Kq"]<0.1,["structid","chain","R","N","Kz","Kq"]]